In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import datetime
import numpy as np
import time

In [2]:
for p,d,f in os.walk('.'):
    print(f)

['close.csv', 'close_check.ipynb', 'data_check.ipynb', 'financials.csv', 'list_check.ipynb', 'options.csv', 'options_check.ipynb', 'options_check_r1.ipynb', 'price_check.ipynb', 'secondary_stock_prices.csv', 'stock_list.csv', 'stock_prices.csv', 'trades.csv', 'シャープ値_業種間の差調査.ipynb', 'シャープ値_相関の強い特徴量探す.ipynb']
['close_check-checkpoint.ipynb', 'data_check-checkpoint.ipynb', 'list_check-checkpoint.ipynb', 'options_check-checkpoint.ipynb', 'options_check_r1-checkpoint.ipynb', 'price_check-checkpoint.ipynb', 'シャープ値_業種間の差調査-checkpoint.ipynb', 'シャープ値_相関の強い特徴量探す-checkpoint.ipynb']


In [3]:
f_list = ['financials.csv', 'options.csv', 'secondary_stock_prices.csv', 'stock_prices.csv', 'trades.csv']

In [4]:
#各ファイルの内容をデータフレームに格納
df_dict = {}
for f in f_list:
    tmp = pd.read_csv(f)
    df_dict[f[:-4]] =  tmp

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,15,16,17,18,19,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df_sp = df_dict['stock_prices']
df_fi = df_dict['financials']
df_op = df_dict['options']
df_sp2 = df_dict['secondary_stock_prices']
df_tr = df_dict['trades']

In [6]:
df_op.head(100)

,DateCode,Date,OptionsCode,WholeDayOpen,WholeDayHigh,WholeDayLow,WholeDayClose,NightSessionOpen,NightSessionHigh,NightSessionLow,...,Putcall,LastTradingDay,SpecialQuotationDay,SettlementPrice,TheoreticalPrice,BaseVolatility,ImpliedVolatility,InterestRate,DividendRate,Dividend
0,20170104_132010018,2017-01-04,132010018,650.0,650.0,480.0,480.0,0.0000,0.0000,0.0000,...,1,20170112,20170113,480.0,478.4587,17.4736,17.5865,0.0091,0.0000,0.0
1,20170104_132010118,2017-01-04,132010118,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,1,20170112,20170113,575.0,571.1385,17.4736,16.5000,0.0091,0.0000,0.0
2,20170104_132010218,2017-01-04,132010218,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,1,20170112,20170113,680.0,677.3710,17.4736,15.8644,0.0091,0.0000,0.0
3,20170104_132010318,2017-01-04,132010318,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,1,20170112,20170113,795.0,791.0383,17.4736,15.2288,0.0091,0.0000,0.0
4,20170104_132010518,2017-01-04,132010518,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,1,20170112,20170113,910.0,909.9947,17.4736,14.5932,0.0091,0.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20170104_132025018,2017-01-04,132025018,18.0,19.0,8.0,8.0,18.0000,19.0000,17.0000,...,1,20170209,20170210,8.0,8.0000,17.4736,36.9308,0.0300,0.0752,0.0
96,20170104_132025218,2017-01-04,132025218,21.0,22.0,10.0,10.0,21.0000,22.0000,20.0000,...,1,20170209,20170210,11.0,10.0785,17.4736,36.0145,0.0300,0.0752,0.0
97,20170104_132025318,2017-01-04,132025318,23.0,26.0,11.0,11.0,23.0000,26.0000,23.0000,...,1,20170209,20170210,11.0,11.0000,17.4736,35.4169,0.0300,0.0752,0.0
98,20170104_132025518,2017-01-04,132025518,26.0,27.0,12.0,12.0,26.0000,27.0000,25.0000,...,1,20170209,20170210,12.0,12.0001,17.4736,34.8151,0.0300,0.0752,0.0


In [7]:
#①清算日までの残り日数：DaysDeltaのカラム追加
#②put/call でスライス
#③exercise priseでスライスを基準にしたn月先”の数値データに変換

In [8]:
#①-1特徴量追加：清算日までの残り時間'DaysDelta'
def daysdelta(df):
    td_list = []
    for today,finish in zip(df['Date'],df['LastTradingDay']):
        #日付（文字列）をdatetimeに変換
        s = datetime.datetime.strptime(today,'%Y-%m-%d')
        #日付(int)をdatetimeに変換
        e = pd.to_datetime(str(finish))
        td = e - s
        td_list.append(td.days)
    df['DaysDelta'] = td_list

daysdelta(df_op)

In [9]:
#①-2特徴量追加：当日の価格変動'Diff'
df_op['Diff'] = df_op['WholeDayClose']-df_op['WholeDayOpen']

In [10]:
#①-3特徴量追加：清算価格/残り日数'PRatio'
def pratio(df):
    p = df['TheoreticalPrice']
    d = df['DaysDelta']
    
    #価格の正規化
    p_max = p.max()
    p_min = p.min()
    p_std = (p-p_min)/(p_max-p_min)
    
    #日数の正規化
    d_max = d.max()
    d_min = d.min()
    d_std = (d-d_min)/(d_max-d_min)
    
    pratio = p_std/d_std
    df['PRatio'] = pratio

In [11]:
#②put/call　でスライス

#OptionsCodeの特定の情報に合致する行を抜き出す
def extract(a,b,c,df,ls):
    #a:スライス始め　b:スライス終わり　c:検索式　
    for i in df['OptionsCode']:
        if str(i)[a:b] == '{}'.format(c[0]) or \
            str(i)[a:b] == '{}'.format(c[1]) :
            ls.append(True)
        else:
            ls.append(False)
    return ls

#訓練データは現物のみ。putとcallをそれぞれ抽出
#〇　put/spot
ps_list = []
extract(1,2,[3,8],df_op,ps_list)
df_ps = df_op[ps_list]

#〇　call/spot
cs_list = []
extract(1,2,[4,9],df_op,cs_list)
df_cs = df_op[cs_list]

In [12]:
#③-1銘柄：exercise priseでスライス
def code_extract(df):
    brand_list = []
    for i in df['OptionsCode']:
        brand_list.append(str(i)[-4:-2])
    brand_list = list(set(brand_list))
    return brand_list

#銘柄リストを作成
brand_list = code_extract(df_op)
brand_list.sort()
print(brand_list)

['00', '01', '02', '03', '05', '06', '07', '08', '10', '11', '12', '13', '15', '16', '17', '18', '20', '21', '22', '23', '25', '26', '27', '28', '30', '31', '32', '33', '35', '36', '37', '38', '40', '41', '42', '43', '45', '46', '47', '48', '50', '51', '52', '53', '55', '56', '57', '58', '60', '61', '62', '63', '65', '66', '67', '68', '70', '71', '72', '73', '75', '76', '77', '78', '80', '81', '82', '83', '85', '86', '87', '88', '90', '91', '92', '93', '95', '96', '97', '98']


In [13]:
#③-2　put/callそれぞれ銘柄毎に抽出
ps_dict = {}
cs_dict = {}

#df_opから銘柄リストを抜粋したDFを作成する関数
def df_extract(df,code):
    cd_mask = []
    code = int(code)
    cd_mask = extract(5,7,[code,code],df,cd_mask)
    cd_df = df.copy()
    return cd_df[cd_mask]

#以下の処理は時間がかかるので途中経過を表示
start_time = time.time()
count = 1
n = len(brand_list)

#銘柄リストを参照して、put/callそれぞれ辞書を作成
for i in brand_list:
    ps_dict[i] = df_extract(df_ps,i)
    cs_dict[i] = df_extract(df_cs,i)
    
    #1イテレート毎の処理時間を計算、表示
    end_time = time.time()
    timedelta = round(end_time - start_time,0)
    print('{}/{} {}'.format(count,n,timedelta))
    count+=1  

1/80 8.0
2/80 18.0
3/80 27.0
4/80 35.0
5/80 44.0
6/80 56.0
7/80 66.0
8/80 76.0
9/80 85.0
10/80 94.0
11/80 101.0
12/80 109.0
13/80 116.0
14/80 124.0
15/80 131.0
16/80 138.0
17/80 145.0
18/80 152.0
19/80 160.0
20/80 167.0
21/80 176.0
22/80 183.0
23/80 193.0
24/80 201.0
25/80 209.0
26/80 219.0
27/80 230.0
28/80 242.0
29/80 250.0
30/80 259.0
31/80 267.0
32/80 276.0
33/80 288.0
34/80 297.0
35/80 308.0
36/80 319.0
37/80 329.0
38/80 340.0
39/80 350.0
40/80 361.0
41/80 372.0
42/80 382.0
43/80 391.0
44/80 398.0
45/80 407.0
46/80 415.0
47/80 423.0
48/80 432.0
49/80 442.0
50/80 453.0
51/80 462.0
52/80 472.0
53/80 482.0
54/80 491.0
55/80 502.0
56/80 513.0
57/80 524.0
58/80 536.0
59/80 544.0
60/80 553.0
61/80 561.0
62/80 569.0
63/80 577.0
64/80 585.0
65/80 594.0
66/80 605.0
67/80 616.0
68/80 629.0
69/80 639.0
70/80 650.0
71/80 660.0
72/80 670.0
73/80 677.0
74/80 687.0
75/80 696.0
76/80 707.0
77/80 719.0
78/80 732.0
79/80 744.0
80/80 754.0


In [14]:
ps_dict.keys()

dict_keys(['00', '01', '02', '03', '05', '06', '07', '08', '10', '11', '12', '13', '15', '16', '17', '18', '20', '21', '22', '23', '25', '26', '27', '28', '30', '31', '32', '33', '35', '36', '37', '38', '40', '41', '42', '43', '45', '46', '47', '48', '50', '51', '52', '53', '55', '56', '57', '58', '60', '61', '62', '63', '65', '66', '67', '68', '70', '71', '72', '73', '75', '76', '77', '78', '80', '81', '82', '83', '85', '86', '87', '88', '90', '91', '92', '93', '95', '96', '97', '98'])

In [15]:
ps_dict['18']

,DateCode,Date,OptionsCode,WholeDayOpen,WholeDayHigh,WholeDayLow,WholeDayClose,NightSessionOpen,NightSessionHigh,NightSessionLow,...,SpecialQuotationDay,SettlementPrice,TheoreticalPrice,BaseVolatility,ImpliedVolatility,InterestRate,DividendRate,Dividend,DaysDelta,Diff
178009,20170509_132051818,2017-05-09,132051818,0.0,0.0,0.0,0.0,-,-,-,...,20170512,2040.0,2031.9983,13.53750,28.9135,0.0082,0.000,0.0,2,0.0
178072,20170509_132061818,2017-05-09,132061818,0.0,0.0,0.0,0.0,-,-,-,...,20170609,2040.0,2031.4426,13.53750,1.0000,0.0300,0.000,0.0,30,0.0
178135,20170509_132071818,2017-05-09,132071818,0.0,0.0,0.0,0.0,-,-,-,...,20170714,2110.0,2102.6714,13.53750,16.0096,0.0464,0.726,0.0,65,0.0
180135,20170510_132051818,2017-05-10,132051818,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,20170512,1980.0,1974.9035,12.52245,31.8742,0.0082,0.000,0.0,1,0.0
180198,20170510_132061818,2017-05-10,132061818,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,20170609,1980.0,1977.1154,12.52245,14.1008,0.0300,0.000,0.0,29,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3545601,20211129_136121818,2021-11-29,136121818,0.0,0.0,0.0,0.0,0,0,0,...,20211210,3660.0,3592.9221,28.08885,45.7268,-0.0018,0.000,0.0,10,0.0
3550093,20211130_136121818,2021-11-30,136121818,0.0,0.0,0.0,0.0,0,0,0,...,20211210,4410.0,4153.9905,29.07165,83.8149,-0.0018,0.000,0.0,9,0.0
3554593,20211201_136121818,2021-12-01,136121818,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,20211210,3700.0,3981.1957,25.84970,1.0000,-0.0018,0.000,0.0,8,0.0
3559131,20211202_136121818,2021-12-02,136121818,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,20211210,4130.0,4123.0172,27.91395,41.9932,-0.0018,0.000,0.0,7,0.0


In [16]:
ps_dict.items()

dict_items([('00', Empty DataFrame
Columns: [DateCode, Date, OptionsCode, WholeDayOpen, WholeDayHigh, WholeDayLow, WholeDayClose, NightSessionOpen, NightSessionHigh, NightSessionLow, NightSessionClose, DaySessionOpen, DaySessionHigh, DaySessionLow, DaySessionClose, TradingVolume, OpenInterest, TradingValue, ContractMonth, StrikePrice, WholeDayVolume, Putcall, LastTradingDay, SpecialQuotationDay, SettlementPrice, TheoreticalPrice, BaseVolatility, ImpliedVolatility, InterestRate, DividendRate, Dividend, DaysDelta, Diff]
Index: []

[0 rows x 33 columns]), ('01', Empty DataFrame
Columns: [DateCode, Date, OptionsCode, WholeDayOpen, WholeDayHigh, WholeDayLow, WholeDayClose, NightSessionOpen, NightSessionHigh, NightSessionLow, NightSessionClose, DaySessionOpen, DaySessionHigh, DaySessionLow, DaySessionClose, TradingVolume, OpenInterest, TradingValue, ContractMonth, StrikePrice, WholeDayVolume, Putcall, LastTradingDay, SpecialQuotationDay, SettlementPrice, TheoreticalPrice, BaseVolatility, Imp

In [17]:
cs_dict.items()

dict_items([('00', Empty DataFrame
Columns: [DateCode, Date, OptionsCode, WholeDayOpen, WholeDayHigh, WholeDayLow, WholeDayClose, NightSessionOpen, NightSessionHigh, NightSessionLow, NightSessionClose, DaySessionOpen, DaySessionHigh, DaySessionLow, DaySessionClose, TradingVolume, OpenInterest, TradingValue, ContractMonth, StrikePrice, WholeDayVolume, Putcall, LastTradingDay, SpecialQuotationDay, SettlementPrice, TheoreticalPrice, BaseVolatility, ImpliedVolatility, InterestRate, DividendRate, Dividend, DaysDelta, Diff]
Index: []

[0 rows x 33 columns]), ('01', Empty DataFrame
Columns: [DateCode, Date, OptionsCode, WholeDayOpen, WholeDayHigh, WholeDayLow, WholeDayClose, NightSessionOpen, NightSessionHigh, NightSessionLow, NightSessionClose, DaySessionOpen, DaySessionHigh, DaySessionLow, DaySessionClose, TradingVolume, OpenInterest, TradingValue, ContractMonth, StrikePrice, WholeDayVolume, Putcall, LastTradingDay, SpecialQuotationDay, SettlementPrice, TheoreticalPrice, BaseVolatility, Imp

In [18]:
df_sp['SecuritiesCode']

0          1301
1          1332
2          1333
3          1376
4          1377
           ... 
2332526    9990
2332527    9991
2332528    9993
2332529    9994
2332530    9997
Name: SecuritiesCode, Length: 2332531, dtype: int64

In [19]:
df_nk = ps_dict['82']

In [20]:
nk_cd =list(df_nk['OptionsCode'].unique())

In [21]:
nk_dict = {}
for cd in nk_cd:
    nk_dict[cd] = df_nk[df_nk['OptionsCode']==cd]